In [1]:
# #@title ##← Click on the circled arrow and wait for up to 1 minute
# !pip install -q transformers >/dev/null

# import pandas as pd, numpy as np
# import os
# import urllib.request
# from transformers import CLIPProcessor, CLIPTextModel, CLIPModel, logging

# from IPython.display import display, Markdown, HTML, clear_output
# import ipywidgets as widgets

# logging.get_verbosity = lambda: logging.NOTSET
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# clear_output()

# def compute_text_embeddings(list_of_strings):
#     inputs = processor(text=list_of_strings, return_tensors="pt", padding=True)
#     return model.get_text_features(**inputs)

# urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1bt1O-iArKuU9LGkMV1zUPTEHZk8k7L65', 'data.csv')
# urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=19aVnFBY-Rc0-3VErF_C7PojmWpBsb5wk', 'data2.csv')
# urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1onKr-pfWb4l6LgL-z8WDod3NMW-nIJxE', 'embeddings.npy')
# urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1KbwUkE0T8bpnHraqSzTeGGV4-TZO_CFB', 'embeddings2.npy')

# df = {0: pd.read_csv('data.csv'), 1: pd.read_csv('data2.csv')}
# embeddings = {0: np.load('embeddings.npy'), 1: np.load('embeddings2.npy')}
# for k in [0, 1]:
#   embeddings[k] = np.divide(embeddings[k], np.sqrt(np.sum(embeddings[k]**2, axis=1, keepdims=True)))
# source = {0: '\nSource: Unsplash', 1: '\nSource: The Movie Database (TMDB)'}

# def get_html(url_list, height=200):
#     html = "<div style='margin-top: 20px; display: flex; flex-wrap: wrap; justify-content: space-evenly'>"
#     for url, title, link in url_list:
#         html2 = f"<img title='{title}' style='height: {height}px; margin-bottom: 10px' src='{url}'>"
#         if len(link) > 0:
#             html2 = f"<a href='{link}' target='_blank'>" + html2 + "</a>"
#         html = html + html2
#     html += "</div>"
#     return html

# query = widgets.Text(layout=widgets.Layout(width='400px'))
# dataset =widgets.Dropdown(
#     options=['Unsplash', 'Movies'],
#     value='Unsplash'
# )
# button = widgets.Button(description="Search")
# output = widgets.Output()

# display(widgets.HBox([query, button, dataset],
#                      layout=widgets.Layout(justify_content='center')),
#         output)

# def image_search(query, n_results=24):
#     text_embeddings = compute_text_embeddings([query]).detach().numpy()
#     k = 0 if dataset.value == 'Unsplash' else 1
#     results = np.argsort((embeddings[k]@text_embeddings.T)[:, 0])[-1:-n_results-1:-1]
#     return [(df[k].iloc[i]['path'],
#              df[k].iloc[i]['tooltip'] + source[k],
#              df[k].iloc[i]['link']) for i in results]

# def on_button_clicked(b):
#     if len(query.value) > 0:
#         results = image_search(query.value)
#         output.clear_output()
#         with output:
#             display(HTML(get_html(results)))

# button.on_click(on_button_clicked)
# dataset.observe(on_button_clicked, names='value')

In [2]:
### Model
import sys
# !{sys.executable} -m pip install torch --index-url https://download.pytorch.org/whl/cu118
# !{sys.executable} -m pip install torch==1.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !{sys.executable} -m pip install torch==1.11.0+cu113  --extra-index-url https://download.pytorch.org/whl/cu113
!{sys.executable} -m pip install torch==1.11.0+cu102  --extra-index-url https://download.pytorch.org/whl/cu102



import torch
import torch.nn as nn


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, is_last=False):
        super(Bottleneck, self).__init__()
        self.is_last = is_last
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        preact = out
        out = F.relu(out)
        if self.is_last:
            return out, preact
        else:
            return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, in_channel=2, zero_init_residual=False, final_pool=False):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.final_pool = final_pool

        # batch size 16
        # # num_blocks = [3, 4, 6, 3]
        self.conv1 = nn.Conv2d(in_channel, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        if self.final_pool:
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves
        # like an identity. This improves the model by 0.2~0.3% according to:
        # https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, num_blocks, stride, maxpool=False, maxpool2=False):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for i in range(num_blocks):
            stride = strides[i]
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
            if maxpool2:
                layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        return nn.Sequential(*layers)

    def forward(self, x, mask=None):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out1 = self.layer4(out)
        if self.final_pool:
            if mask is not None:
                # mask penultimate features out1
                # in other words, average over only the relavant portions of the image
                out1[~mask.expand_as(out1)] = 0

            out = self.avgpool(out1)
            out = torch.flatten(out, 1)
            return out
        return out1

    
class MatchingModel(nn.Module):
    """backbone + projection head"""
    def __init__(self, feat_dim=128, in_channel=2, dim_in=2048):
        super(MatchingModel, self).__init__()
        self.encoder = ResNet(Bottleneck, [3, 4, 6, 3], in_channel=in_channel, final_pool=False)
        self.head = nn.Sequential(
                nn.Conv2d(dim_in, dim_in, 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(dim_in, feat_dim, 1)
            )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x, spatial_feat=False, mask=None):
        feat = self.encoder(x, mask=mask)
        if mask is not None:
            embedded = self.head(feat)
            embedded[~mask.expand_as(embedded)] = 0
        else:
            embedded = self.head(feat)

        return embedded if spatial_feat else self.vectorize(embedded) # , _
    def vectorize(self, x):
        return F.normalize(torch.flatten(self.avgpool(x), 1), dim=1)



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu102


In [3]:
import requests

def download(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)   
    return response

def get_confirm_token(response):
    if 'too large for Google to scan for viruses. Would you still like to download this file?' in response.text:
        return True
#     for key, value in response.cookies.items():
#         if key.startswith('download_warning'):
#             return value

#     return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [7]:
#@title ##← Click on the circled arrow and wait for up to 1 minute
# !pip install -q transformers >/dev/null

# import pandas as pd, 
import numpy as np
import os
import urllib.request
# from transformers import CLIPProcessor, CLIPTextModel, CLIPModel, logging

from IPython.display import display, Markdown, HTML, clear_output
import ipywidgets as widgets
import io
from PIL import Image

# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# logging.get_verbosity = lambda: logging.NOTSET

clear_output()

# def compute_text_embeddings(list_of_strings):
#     inputs = processor(text=list_of_strings, return_tensors="pt", padding=True)
#     return model.get_text_features(**inputs)


device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')    
net = MatchingModel().to(device)
print('==> Loading model')
model_name = 'model.pth'
if not os.path.exists(model_name):
    file_id = '1XHilLyTOpdM0IRI3NmVPjSFyb_Vgi4I_'
    destination = model_name
    response = download(file_id, destination)
file = torch.load(model_name)
# model.load_state_dict(file['model'])

database_feat_name = 'database_features.pt'
if not os.path.exists(database_feat_name):
    file_id = '1qC341oZLESYO-hc3H2efhz5w7YDqag6s'
    destination = database_feat_name
    response = download(file_id, destination)

# df = {0: pd.read_csv('data.csv'), 1: pd.read_csv('data2.csv')}
# embeddings = {0: np.load('embeddings.npy'), 1: np.load('embeddings2.npy')}
# for k in [0, 1]:
#   embeddings[k] = np.divide(embeddings[k], np.sqrt(np.sum(embeddings[k]**2, axis=1, keepdims=True)))
# source = {0: '\nSource: Unsplash', 1: '\nSource: The Movie Database (TMDB)'}

# def get_html(url_list, height=200):
#     html = "<div style='margin-top: 20px; display: flex; flex-wrap: wrap; justify-content: space-evenly'>"
#     for url, title, link in url_list:
#         html2 = f"<img title='{title}' style='height: {height}px; margin-bottom: 10px' src='{url}'>"
#         if len(link) > 0:
#             html2 = f"<a href='{link}' target='_blank'>" + html2 + "</a>"
#         html = html + html2
#     html += "</div>"
#     return html

# query = widgets.Text(layout=widgets.Layout(width='400px'))
# dataset =widgets.Dropdown(
#     options=['Unsplash', 'Movies'],
#     value='Unsplash'
# )
button = widgets.Button(description="Search")
output = widgets.Output()
uploader = widgets.FileUpload()
uploader

# display(widgets.HBox([uploader, button],
#                      layout=widgets.Layout(justify_content='center')),
#         output)

def image_search(query, n_results=24):
    text_embeddings = compute_text_embeddings([query]).detach().numpy()
    k = 0 if dataset.value == 'Unsplash' else 1
    results = np.argsort((embeddings[k]@text_embeddings.T)[:, 0])[-1:-n_results-1:-1]
    return [(df[k].iloc[i]['path'],
             df[k].iloc[i]['tooltip'] + source[k],
             df[k].iloc[i]['link']) for i in results]

def on_button_clicked(b):
    for name, file_info in uploader.value.items():
        img = Image.open(io.BytesIO(file_info['content']))
        print(img)
    
#         results = image_search(query.value)
#         output.clear_output()
#         with output:
#             display(HTML(get_html(results)))

button.on_click(on_button_clicked)
# dataset.observe(on_button_clicked, names='value')

==> Loading model


ModuleNotFoundError: No module named 'codes'

In [6]:
import sys
# !{sys.executable} -m pip install torch --index-url https://download.pytorch.org/whl/cu118
# !{sys.executable} -m pip uninstall torch
# !{sys.executable} -m pip install torch==1.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html


import torch
print(torch.cuda.is_available())
# model_name
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu') 
print(device)
print(model_name)
file = torch.load(model_name, map_location=device)
# file = torch.load(model_name)

True
cuda:1
model.pth


ModuleNotFoundError: No module named 'codes'

In [23]:
# for name, file_info in uploader.value.items():
#     with open(name, 'wb') as fp:
#         fp.write(file_info['content'])
        
import io
from PIL import Image

for name, file_info in uploader.value.items():
    img = Image.open(io.BytesIO(file_info['content']))
    
print(img)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x630 at 0x7FCE3E1DE5D0>


In [20]:
file = torch.load(model_name)

ModuleNotFoundError: No module named 'codes'

In [23]:
model_name


'model.pth'

In [24]:
torch.load(model_name)

ModuleNotFoundError: No module named 'codes'